<a href="https://colab.research.google.com/github/kingketan9/Movie-Recomendation-System/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MOVIE RECOMMENDATION SYSTEM

**Importing the libraries**

In [5]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity

**Loading the dataset**

In [6]:
movies_data = pd.read_csv('/content/movies.csv')

**Data Collection & Analysis**

In [7]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [8]:
movies_data.tail()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,"[{'name': 'Daniel Hsia', 'gender': 2, 'departm...",Daniel Hsia
4802,4802,0,Documentary,NaN,25975,obsession camcorder crush dream girl,en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,My Date with Drew,6.3,16,Drew Barrymore Brian Herzlinger Corey Feldman ...,"[{'name': 'Clark Peterson', 'gender': 2, 'depa...",Brian Herzlinger


In [9]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [10]:
movies_data.shape

(4803, 24)

In [11]:
movies_data.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [12]:
# selecting the relevant features for the target variable
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


**Filling the null values with null string**

In [13]:
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [14]:
# combining all the selected features
combined_features = movies_data['genres'] + ' ' + movies_data['keywords'] + ' ' + movies_data['tagline'] + ' ' + movies_data['cast'] + ' ' + movies_data['director'] 

In [15]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


**Converting the text data into feature vectors**

In [16]:
vectorizer = TfidfVectorizer()

In [17]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [18]:
print(feature_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

**Getting similarity scores using cosine similarity**

In [19]:
similarity = cosine_similarity(feature_vectors)

In [20]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [21]:
print(similarity.shape)

(4803, 4803)


**Getting an input from the user**

In [22]:
movie_name = input('Enter the name of the movie : ')

Enter the name of the movie : The Ultimate Gift


In [23]:
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

**Finding the close match for the movie name given by user**

In [24]:
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['The Ultimate Gift', 'The Gift', 'The Hit List']


In [25]:
close_match = find_close_match[0]
print(close_match)

The Ultimate Gift


In [26]:
index_of_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_movie)

3183


**Getting a list of similar movies**

In [27]:
# enumerate just basically runs the loop in a list 
similarity_score = list(enumerate(similarity[index_of_movie]))
print(similarity_score)
# 0, 1 , 2... --> index of the movie
# and the other value just represents the similarity score

[(0, 0.013409361387527942), (1, 0.0), (2, 0.0), (3, 0.01637233913448554), (4, 0.0), (5, 0.01389104856045652), (6, 0.010095843511706904), (7, 0.0), (8, 0.021993897004052514), (9, 0.0), (10, 0.01508244447331206), (11, 0.0), (12, 0.023420738306178555), (13, 0.012361651305994724), (14, 0.011245784113624606), (15, 0.02434674297773472), (16, 0.0), (17, 0.0), (18, 0.010814929508170548), (19, 0.0), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.0), (24, 0.048921560312524474), (25, 0.02486766642615595), (26, 0.0), (27, 0.026032320154636912), (28, 0.0), (29, 0.0), (30, 0.013514618559668404), (31, 0.0), (32, 0.009228273732065917), (33, 0.0), (34, 0.01011664850661499), (35, 0.011919510765625372), (36, 0.010740430398396306), (37, 0.024587305233549576), (38, 0.0), (39, 0.0), (40, 0.02256204704658881), (41, 0.0), (42, 0.02127759266608399), (43, 0.0), (44, 0.01445400747007408), (45, 0.05755641042777659), (46, 0.023770946015509574), (47, 0.0), (48, 0.010220300840401857), (49, 0.009447140882792329), (50, 0.007

In [28]:
len(similarity_score)

4803

**Sorting the movies based on their similarity score**

In [29]:
# sort the movies based on the similarity score from decreasing order
# x --> similarity score 
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(3183, 1.0), (3363, 0.18711384487889562), (2913, 0.1708498554893456), (488, 0.14219301386231117), (3238, 0.14000484334231458), (3857, 0.1369791448901166), (3462, 0.13621503667430285), (3452, 0.12279283963690957), (1331, 0.11225032362551447), (1745, 0.11034340434648228), (1579, 0.110033477726793), (4640, 0.10911062515862782), (2270, 0.10614424083567028), (1988, 0.10496595765751383), (3670, 0.10252471078606848), (3742, 0.1018350857344005), (725, 0.1006832636449864), (3900, 0.10051467926936357), (2362, 0.09832352347345524), (1559, 0.0957002751405378), (2202, 0.09445355470173517), (3345, 0.09409757004790406), (564, 0.09341846907087673), (3125, 0.09301896031387479), (1719, 0.0908904515115011), (2783, 0.09074176583778908), (260, 0.09071601379643751), (553, 0.09043674807524879), (3327, 0.08982654611371337), (3026, 0.0865866991161252), (4730, 0.08484036038740507), (2800, 0.08482929419549146), (3625, 0.08472818246854143), (1751, 0.08365030301450181), (2595, 0.08094886994037247), (3872, 0.07981

In [30]:
print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index == index]['title'].values[0]
  if (i < 30):
    print(i, '.', title_from_index)
    i = i + 1

Movies suggested for you : 

1 . The Ultimate Gift
2 . Get Low
3 . Dirty Grandpa
4 . Arthur and the Invisibles
5 . Little Miss Sunshine
6 . Wicked Blood
7 . Daddy Day Camp
8 . The Brothers
9 . Nim's Island
10 . My Sister's Keeper
11 . Midnight in Paris
12 . Love and Other Catastrophes
13 . Zambezia
14 . Zombieland
15 . Running Forever
16 . Maggie
17 . The Shaggy Dog
18 . Air Bud
19 . And So It Goes
20 . The Notebook
21 . One Night with the King
22 . Definitely, Maybe
23 . Signs
24 . Eulogy
25 . No Reservations
26 . The Call
27 . Ender's Game
28 . The Kingdom
29 . Ta Ra Rum Pum


# Building a predictive system

In [31]:
movie_name = input('Enter your favourite movie name here : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i = i + 1

Enter your favourite movie name here : John Wick
Movies suggested for you : 

1 . John Q
2 . The Great Debaters
3 . He Got Game
4 . Phone Booth
5 . The Judge
6 . Dear Frankie
7 . The Manchurian Candidate
8 . The Juror
9 . Diary of a Mad Black Woman
10 . Mission: Impossible - Ghost Protocol
11 . Gangster Squad
12 . The Astronaut's Wife
13 . When Did You Last See Your Father?
14 . 2016: Obama's America
15 . Four Christmases
16 . Containment
17 . We Own the Night
18 . Wag the Dog
19 . Clerks II
20 . Fireproof
21 . Antwone Fisher
22 . True Romance
23 . Snitch
24 . Evan Almighty
25 . Safe House
26 . The Shaggy Dog
27 . Rudderless
28 . Man on Fire
29 . Open Range
